In [1]:
import numpy as np
import pandas as pd
import pyreadr
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

C:\Users\rshoo\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\rshoo\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\rshoo\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [8]:
data = pyreadr.read_r('C:\\Users\\rshoo\\OneDrive\\Desktop\\Rush Interview\\labelled_data_with_pcs.RDS')[None]
gender = {'M': 1,'F': 0}
data.sex = [gender[i] for i in data.sex]
data.dropna(axis = 0, how = 'any', inplace = True)
data.head()

,cohorttype,sex,age,post911,branch,ethnic,race,married,education,deployed,...,pcl5,psycPC1,psycPC2,psycPC3,demoPC1,demoPC2,demoPC3,totalPC1,totalPC2,totalPC3
0,1,0,43,0,0,0,0,0,2,1,...,3,-1.679615,2.215430,0.464686,1.599222,0.465677,1.323373,2.157840,1.845729,-1.052714
1,0,1,34,0,0,0,1,1,4,0,...,4,-0.140718,-2.036487,-1.281802,-2.576349,-1.017835,-0.053567,-0.627375,-3.391069,0.335486
2,1,0,37,0,0,0,1,0,5,1,...,3,-0.736406,0.340951,0.799209,1.066885,0.418891,-1.039251,1.000772,1.232303,0.400928
3,1,0,47,1,0,0,0,0,6,1,...,2,-3.785656,-0.745756,1.028510,0.207235,2.580869,0.608060,3.781963,-0.188842,2.083140
4,1,1,44,0,0,0,0,2,7,1,...,2,0.447259,-0.432621,-2.431265,-0.471273,-0.727869,-0.753474,-0.657175,-1.217646,-1.337110


In [19]:
k=10
predictors=data.drop('responder',axis=1,inplace=False)
response= data[['responder']]
rank=20
repeated_misclass_list=[]
for r in range(1,rank):
    rn = range(1,data.shape[0])
    fold = KFold(n_splits=k, shuffle=True)
    misclass_list=[]
    for train_index, test_index in fold.split(rn):
        X_holdin=predictors.iloc[train_index]
        Y_holdin=response.iloc[train_index]
        X_holdout=predictors.iloc[test_index]
        Y_holdout=response.iloc[test_index]

        scaler = StandardScaler()
        scaler.fit(X_holdin)
        scaled=scaler.transform(X_holdin)

        n_cols = predictors.shape[1]                                                
        model = Sequential()
        model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
        model.add(Dropout(0.2))
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(25, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1,activation="sigmoid"))

        model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
        model.fit(scaled, Y_holdin,epochs=30, batch_size=100)

        scaler = StandardScaler()
        scaler.fit(X_holdout)
        holdout_scaled=scaler.transform(X_holdout)

        predictions=(model.predict(holdout_scaled) > 0.5).astype("int32")

        confusion=confusion_matrix(Y_holdout,predictions)
        misclass=1 - confusion.diagonal().sum() / confusion.sum()
        misclass_list.append(misclass)
    repeated_misclass_list.append(sum(misclass_list)/len(misclass_list))

Epoch 1/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6847 - accuracy: 0.5931
Epoch 2/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6419 - accuracy: 0.6528
Epoch 3/30
4/4 [==============================] - 0s 4ms/step - loss: 0.6173 - accuracy: 0.6666
Epoch 4/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5883 - accuracy: 0.7097
Epoch 5/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6030 - accuracy: 0.7126
Epoch 6/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5588 - accuracy: 0.7209
Epoch 7/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5640 - accuracy: 0.7113
Epoch 8/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5585 - accuracy: 0.7391
Epoch 9/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5565 - accuracy: 0.7354
Epoch 10/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5564 - accuracy: 0.7285
Epoch 11/30
4/4 [======================

4/4 [==============================] - 0s 4ms/step - loss: 0.5888 - accuracy: 0.6936
Epoch 10/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5908 - accuracy: 0.7142
Epoch 11/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5692 - accuracy: 0.7081
Epoch 12/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5770 - accuracy: 0.7307
Epoch 13/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5243 - accuracy: 0.7442
Epoch 14/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5333 - accuracy: 0.7382
Epoch 15/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5173 - accuracy: 0.7819
Epoch 16/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5292 - accuracy: 0.7464
Epoch 17/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5264 - accuracy: 0.7759
Epoch 18/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5362 - accuracy: 0.7157
Epoch 19/30
4/4 [=========================

4/4 [==============================] - 0s 2ms/step - loss: 0.5081 - accuracy: 0.7585
Epoch 19/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5072 - accuracy: 0.7554
Epoch 20/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5257 - accuracy: 0.7094
Epoch 21/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5107 - accuracy: 0.7666
Epoch 22/30
4/4 [==============================] - 0s 2ms/step - loss: 0.4961 - accuracy: 0.7556
Epoch 23/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5156 - accuracy: 0.7368
Epoch 24/30
4/4 [==============================] - 0s 2ms/step - loss: 0.4828 - accuracy: 0.7646
Epoch 25/30
4/4 [==============================] - 0s 2ms/step - loss: 0.4790 - accuracy: 0.7541
Epoch 26/30
4/4 [==============================] - 0s 2ms/step - loss: 0.4782 - accuracy: 0.7473
Epoch 27/30
4/4 [==============================] - 0s 3ms/step - loss: 0.4867 - accuracy: 0.7589
Epoch 28/30
4/4 [=========================

4/4 [==============================] - 0s 3ms/step - loss: 0.5102 - accuracy: 0.7586
Epoch 28/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5082 - accuracy: 0.7447
Epoch 29/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5025 - accuracy: 0.7613
Epoch 30/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5225 - accuracy: 0.7524
Epoch 1/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.5775
Epoch 2/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6483 - accuracy: 0.6247
Epoch 3/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6271 - accuracy: 0.6515
Epoch 4/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5915 - accuracy: 0.7052
Epoch 5/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5926 - accuracy: 0.7086
Epoch 6/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5936 - accuracy: 0.7011
Epoch 7/30
4/4 [==============================] 

Epoch 1/30
4/4 [==============================] - 1s 2ms/step - loss: 0.7417 - accuracy: 0.4392
Epoch 2/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6768 - accuracy: 0.5467
Epoch 3/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6378 - accuracy: 0.6617
Epoch 4/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6350 - accuracy: 0.6600
Epoch 5/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6150 - accuracy: 0.7041
Epoch 6/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6461 - accuracy: 0.6724
Epoch 7/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5957 - accuracy: 0.6889
Epoch 8/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5702 - accuracy: 0.7047
Epoch 9/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6006 - accuracy: 0.6897
Epoch 10/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5883 - accuracy: 0.7129
Epoch 11/30
4/4 [======================

...

4/4 [==============================] - 0s 4ms/step - loss: 0.6227 - accuracy: 0.6659
Epoch 3/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6186 - accuracy: 0.6671
Epoch 4/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6435 - accuracy: 0.6492
Epoch 5/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6198 - accuracy: 0.6697
Epoch 6/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6028 - accuracy: 0.6993
Epoch 7/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5721 - accuracy: 0.7078
Epoch 8/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5591 - accuracy: 0.7141
Epoch 9/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5692 - accuracy: 0.7194
Epoch 10/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.6923
Epoch 11/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5597 - accuracy: 0.7441
Epoch 12/30
4/4 [==============================] 

4/4 [==============================] - 0s 4ms/step - loss: 0.5675 - accuracy: 0.6865
Epoch 11/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5356 - accuracy: 0.7406
Epoch 12/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5061 - accuracy: 0.7626
Epoch 13/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5488 - accuracy: 0.7181
Epoch 14/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5129 - accuracy: 0.7303
Epoch 15/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5356 - accuracy: 0.7156
Epoch 16/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5121 - accuracy: 0.7441
Epoch 17/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5258 - accuracy: 0.7227
Epoch 18/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5177 - accuracy: 0.7411
Epoch 19/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5203 - accuracy: 0.7480
Epoch 20/30
4/4 [=========================

4/4 [==============================] - 0s 3ms/step - loss: 0.5253 - accuracy: 0.7458
Epoch 20/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5383 - accuracy: 0.7539
Epoch 21/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5333 - accuracy: 0.7327
Epoch 22/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5190 - accuracy: 0.7362
Epoch 23/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5360 - accuracy: 0.7173
Epoch 24/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5236 - accuracy: 0.7318
Epoch 25/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5332 - accuracy: 0.7394
Epoch 26/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5074 - accuracy: 0.7529
Epoch 27/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5168 - accuracy: 0.7419
Epoch 28/30
4/4 [==============================] - 0s 3ms/step - loss: 0.4902 - accuracy: 0.7441
Epoch 29/30
4/4 [=========================

4/4 [==============================] - 0s 3ms/step - loss: 0.5066 - accuracy: 0.7797
Epoch 29/30
4/4 [==============================] - 0s 4ms/step - loss: 0.5180 - accuracy: 0.7440
Epoch 30/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5153 - accuracy: 0.7474
Epoch 1/30
4/4 [==============================] - 0s 2ms/step - loss: 0.7958 - accuracy: 0.4512
Epoch 2/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6941 - accuracy: 0.5532
Epoch 3/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6463 - accuracy: 0.6635
Epoch 4/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6285 - accuracy: 0.6409
Epoch 5/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5982 - accuracy: 0.6964
Epoch 6/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5711 - accuracy: 0.7086
Epoch 7/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6009 - accuracy: 0.6993
Epoch 8/30
4/4 [==============================] -

Epoch 1/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6740 - accuracy: 0.6229
Epoch 2/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6535 - accuracy: 0.6381
Epoch 3/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6043 - accuracy: 0.6984
Epoch 4/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6253 - accuracy: 0.6537
Epoch 5/30
4/4 [==============================] - 0s 3ms/step - loss: 0.6185 - accuracy: 0.6876
Epoch 6/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5764 - accuracy: 0.6958
Epoch 7/30
4/4 [==============================] - 0s 2ms/step - loss: 0.6138 - accuracy: 0.7065
Epoch 8/30
4/4 [==============================] - 0s 2ms/step - loss: 0.5654 - accuracy: 0.7293
Epoch 9/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5933 - accuracy: 0.6961
Epoch 10/30
4/4 [==============================] - 0s 3ms/step - loss: 0.5824 - accuracy: 0.7026
Epoch 11/30
4/4 [======================

In [20]:
sum(repeated_misclass_list)/len(repeated_misclass_list)

0.3008099517085396